# Perkolation

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook werden die Knoten- und die Kantenperkolation untersucht, wobei zur Identifikation von Clustern der Hoshen-Kopelman-Algorithmus herangezogen wird. Im ersten Teil des Jupyter-Notebooks wird für eine vorgegebene Besetzungswahrscheinlichkeit die Clustereinteilung eines zufälligen Zustands bestimmt und farblich dargestellt. Im zweiten Teil wird dann die Perkolationswahrscheinlichkeit als Funktion der Besetzungswahrscheinlichkeit untersucht.

## Importanweisungen

Neben bereits bekannten Importanweisungen wird hier auch das `time`-Modul aus der Python-Standardbibliothek importiert, um bei der typischerweise etwas länger dauernden Berechnung der Perkolationswahrscheinlichkeit die Rechenzeit in Abhängigkeit von der Systemgröße im Blick haben zu können.

In [ ]:
from math import isqrt
import time
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Aufteilung eines Zustands in Cluster mit dem Hoshen-Kopelman-Algorithmus

### Erzeugung eines zufälligen Zustands

Für eine vorgegebene Wahrscheinlichkeit `p` wird die Besetzung von `n_total` Gitterplätzen zufällig festgelegt. Da die Besetzung eines Knotens oder einer Kante unabhängig von den benachbarten Knoten bzw. Kanten erfolgt, spielt die Form des Gitters hier keine Rolle. Dem Wahrheitswert `True` im Boole'schen Array `state` entsprechen besetzte Knoten oder Kanten.

In [ ]:
rng = np.random.default_rng()

def generate_state(n_total, p):
    random_numbers = rng.uniform(size=n_total)
    state = (random_numbers < p)
    return state

### Indizes der nächsten Nachbarn

Die Generatorfunktionen `neighbours_sites` und `neighbors_bonds` geben die Indizes der nächsten Nachbarn mit niedrigeren Indizes für Knoten bzw. Kanten auf einem Quadratgitter zurück. Im ersten Fall kann es maximal zwei benachbarte Knoten geben, während sich im zweiten Fall bis zu drei Kanten ergeben. Am Rand kann die Anzahl der nächsten Nachbarn geringer sein.

In [ ]:
def neighbours_sites(n, n_edge):
    n1, n2 = divmod(n, n_edge)
    if n1 > 0:
        yield n-n_edge
    if n2 > 0:
        yield n-1

In [ ]:
def neighbours_bonds(n, n_edge):
    n1, n2 = divmod(n, n_edge)
    if n1 % 2:
        if n2 > 0:
            yield n-n_edge-1
            yield n-1
        yield n-n_edge
    else:
        if n1 > 0:
            yield n-2*n_edge
            yield n-n_edge
            if n2 < n_edge-1:
                yield n-n_edge+1

### Aufteilung in Cluster

Die Aufteilung des Zustands `state` in zusammenhängende Cluster wird mit Hilfe des Hoshen-Kopelman-Algorithmus durchgeführt. Dazu geht man einmal durch das gesamte Gitter und betrachtet besetzte Gitterplätze. Im Set `neighbour_clusters` werden die Clusterindizes von benachbarten besetzten Gitterplätzen gesammelt. Falls dieses Set keine Elemente enthält, wird ein neuer Clusterindex vergeben. Andernfalls geht man zunächst davon aus, dass ein neuer Clusterindex `good_index` vergeben wird, betrachtet dann aber die Indizes der benachbarten Cluster. Im Set `affected_cluster_numbers` werden die Indizes gesammelt, die eventuell umzubennen sind. Anschließend wird `good_index` ggf. durch einen niedrigeren Index im verbundenen Cluster ersetzt. Dann wird noch im Array `indices` für die betroffenen Indizes in `affected_cluster_numbers` der neue Clusterindex `good_index` hinterlegt. Entscheidend ist hier, dass die Aktualisierung des Clusterindex nicht für alle betroffenen Gitterplätze erfolgt, sondern nur im Array `indices` vermerkt wird. Abschließend wird das Array `indices` noch einmal aktualisiert, um die spätere Vereinigung entfernter Cluster zu berücksichtigen. Als Ergebnis werden die vergegebenen Clusterindizes `cluster_numbers` und die Abbildung auf gute Clusterindizes `indices` übergeben. 

In [ ]:
def get_clusters(state, n_edge, neighbours_fct):
    n_total = state.size
    cluster_numbers = np.zeros(n_total, dtype=int)
    indices = np.zeros(n_total, dtype=int)
    next_index = 1

    for n in range(n_total):
        if state[n]:
            neighbour_clusters = set()
            affected_cluster_numbers = set()
            for nbr in neighbours_fct(n, n_edge):
                if cluster_numbers[nbr] > 0:
                    neighbour_clusters.add(
                        cluster_numbers[nbr])
            if len(neighbour_clusters) == 0:
                cluster_numbers[n] = next_index
                indices[next_index] = next_index
                next_index = next_index+1
            else:
                good_index = next_index
                for nbr_cluster in neighbour_clusters:
                    idx = nbr_cluster
                    affected_cluster_numbers.add(idx)
                    while indices[idx] != idx:
                        idx = indices[idx]
                        affected_cluster_numbers.add(idx)
                    good_index = min(good_index, idx)
                cluster_numbers[n] = good_index
            for index in affected_cluster_numbers:
                indices[index] = good_index

    for n in range(next_index):
        indices[n] = indices[indices[n]]
    return cluster_numbers, indices

### Überprüfung auf einen perkolierenden Cluster

Es wird das Auftreten eines perkolierenden Clusters, der den oberen und den unteren Rand des Gitters verbindet, überprüft. Dazu muss zumindest ein Clusterindex sowohl am oberen als auch am unteren Rand vorkommen. Um den korrekten Clusterindex zu erhalten, muss die in der Funktion `get_clusters` erzeugte Abbildung mit Hilfe des Arrays `indices` berücksichtigt werden.

In [ ]:
def isconnected(state, n_edge, cluster_numbers, indices):
    lower_clusters = indices[cluster_numbers[-n_edge:]]
    upper_clusters = indices[cluster_numbers[:n_edge]]
    common_indices = np.intersect1d(upper_clusters,
                                    lower_clusters)
    connected = np.count_nonzero(common_indices)
    return min(1, connected)

### Aufbereitung der Clusterinformation für die graphische Darstellung

Da die Cluster unterschiedlich eingefärbt werden sollen, ist es sinnvoll, Lücken in den vergebenen Clusterindizes zu vermeiden um in den Funktionen `colorize_sites` und `colorize_bonds` eine gleichmäßige Abbildung auf Farben mit Hilfe des Arrays `color_idx` durchführen zu können. Dazu benennt man die Indizes im Array `indices` zunächst mit Hilfe der Funktion `compress` um. Die Abbildung von den alten Clusterindizes auf die neuen Clusterindizes wird dabei im Dictionary `idx_trans` abgespeichert.

In [ ]:
def compress(x):
    new_x = []
    idx_trans = {0: 0}
    idx = 1
    for elem in x:
        try:
            new_x.append(idx_trans[elem])
        except KeyError:
            idx_trans[elem] = idx
            new_x.append(idx)
            idx = idx + 1
    return new_x

Für die Knotenperkolation wird zunächst einmal für jeden Knoten der Wert -1 vergeben, der für belegte Knoten durch den zugehörigen Farbindex `color_idx`, der zwischen 0 und 1 läuft, überschrieben wird. Anschließend werden alle Einträge, die zu unbelegten Knoten gehören, maskiert, so dass später nur die belegten Knoten eingefärbt werden. Um die Cluster farblich besser unterscheiden zu können, ist es sinnvoll, das Array `color_idx` vor der Benutzung mit `rng.shuffle` durchzumischen.

In [ ]:
def colorize_sites(state, cluster_numbers, indices):
    n_total = state.size
    indices = compress(indices)
    largest_idx = max(indices)
    color_idx = np.arange(largest_idx)/largest_idx
    rng.shuffle(color_idx)
    state_for_plot = -np.ones(n_total)
    for n in range(n_total):
        if state[n]:
            cluster_num = indices[cluster_numbers[n]]
            state_for_plot[n] = color_idx[cluster_num-1]
    return np.ma.masked_equal(state_for_plot, -1)

Für die Kantenperkolation ist es günstig, neben den zuvor beschriebenen Schritten auch noch die Anfangs- und Endkoordinaten der Kanten vorzubereiten. Außerdem erfolgt hier schon die Abbildung von den Farbindizes auf Farben mit Hilfe von `cmap`. Bei Interesse kann man die verwendete Farbpalette in `matplotlib.colormaps` anpassen. Die möglichen Farbpaletten findet man in der [Dokumentation zu `matplotlib`](https://matplotlib.org/stable/gallery/color/colormap_reference.html). In der vorigen Funktion `colorize_sites` wird diese Abbildung auf die Farbe noch nicht vorgenommen, da die Farbpalette zur graphischen Darstellung noch weiter modifiziert wird.

In [ ]:
def colorize_bonds(state, cluster_numbers, indices):
    n_total = state.size
    n_edge = isqrt(n_total//2)
    indices = compress(indices)
    largest_idx = max(indices)
    cmap = matplotlib.colormaps["hsv"]
    color_idx = np.arange(largest_idx)/largest_idx
    rng.shuffle(color_idx)
    for n in range(n_total):
        if state[n]:
            cluster_num = indices[cluster_numbers[n]]
            color = cmap(color_idx[cluster_num-1])
            n1, n2 = divmod(n, n_edge)
            if n1 % 2:
                x1 = n2
                x2 = n2+1
                y1 = n_edge-(n1-1)//2-1
                yield x1, x2, y1, y1, color
            else:
                x1 = n2+1
                y1 = n_edge-n1//2
                y2 = n_edge-n1//2-1
                yield x1, x1, y1, y2, color

### Implementierung der Bedienelemente und graphische Darstellung des Zustands

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `n_edge`: Seitenlänge des Gitters
- `p`: Besetzungswahrscheinlichkeit. Die Anfangseinstellung liegt an oder in der Nähe der Perkolationsschwelle.
- `percolation_type`: Auswahl zwischen Knoten- und Kantenperkolation auf dem Quadratgitter

Die Erzeugung der graphischen Darstellung ist in separate Funktionen ausgelagert. Zunächst erzeugt die Funktion `get_state_cluster` mit Hilfe der weiter oben definierten Funktionen einen Zustand und zerlegt diesen in Cluster. Ferner wird untersucht, ob ein perkolierender Cluster vorliegt und eine entsprechende Information ausgegeben. Die farbliche Darstellung erfolgt durch die Funktionen `plot_sites` und `plot_bonds`. Dabei gibt es in der Funktion `plot_sites` noch die Besonderheit, dass die Farbpalette so modifiziert wird, dass ein unerlaubter Wert für die Abbildung auf eine Farbe die Farbe weiß ergibt.

In [ ]:
def get_state_cluster(n_edge, p, n_total, neighbours_fct):
    state = generate_state(n_total, p)
    cluster_numbers, indices = get_clusters(state, n_edge,
                                            neighbours_fct)
    connected = isconnected(state, n_edge,
                            cluster_numbers, indices)
    print(f"{'einen' if connected else 'keinen'} "
          "perkolierenden Cluster gefunden")
    return state, cluster_numbers, indices

In [ ]:
def plot_sites(n_edge, p):
    state, cluster_numbers, indices = get_state_cluster(
        n_edge, p, n_edge**2, neighbours_sites)
    colored_state = np.reshape(
        colorize_sites(state, cluster_numbers, indices),
        (n_edge, n_edge))
    cmap = matplotlib.cm.hsv
    cmap.set_bad("white")
    plt.imshow(colored_state, aspect=1,
               interpolation="none", cmap=cmap)

In [ ]:
def plot_bonds(n_edge, p):
    state, cluster_numbers, indices = get_state_cluster(
        n_edge, p, 2*n_edge**2, neighbours_bonds)
    bonds = colorize_bonds(state, cluster_numbers, indices)
    fig, ax = plt.subplots()
    for x1, x2, y1, y2, color in bonds:
        ax.plot((x1, x2), (y1, y2), color=color)
    ax.set_aspect("equal")
    ax.invert_yaxis()

In [ ]:
widget_dict = {"n_edge":
               widgets.IntSlider(
                   value=100, min=3, max=100, step=1,
                   description=r"$n_\text{edge}$"),
               "p":
               widgets.FloatSlider(
                   value=0.6, min=0, max=1, step=0.01,
                   description="$p$"),
               "percolation_type":
               widgets.RadioButtons(
                   value="Knotenperkolation",
                   options=["Knotenperkolation",
                            "Kantenperkolation"],
                   description="Perkolationstyp")
               }

def handle_type_change(change):
    if change.new == "Knotenperkolation":
        setattr(widget_dict["p"], "value", 0.6)
    else:
        setattr(widget_dict["p"], "value", 0.5)

widget_dict["percolation_type"].observe(handle_type_change,
                                        names="value")

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_realization(n_edge, p, percolation_type):
    if percolation_type == "Knotenperkolation":
        plot_sites(n_edge, p)
    else:
        plot_bonds(n_edge, p)

## Perkolationswahrscheinlichkeit als Funktion von $p$

### Berechnung der Perkolationswahrscheinlichkeit für einen Wert von $p$

Für eine Besetzungswahrscheinlichkeit `p` wird die Perkolationswahrscheinlichkeit durch Untersuchung von `n_ensemble` verschiedenen Zuständen berechnet.

In [ ]:
def probability(p, n_edge, n_ensemble, n_total,
                neighbours_fct):
    incidence = 0
    for n in range(n_ensemble):
        state = generate_state(n_total, p)
        cluster_numbers, indices = get_clusters(
            state, n_edge, neighbours_fct)
        connected = isconnected(state, n_edge,
                                cluster_numbers, indices)
        incidence = incidence + connected
    return incidence / n_ensemble

### Berechnung der Perkolationswahrscheinlichkeit als Funktion von $p$

Unter Benutzung der vorigen Funktion `probability` wird nun die Perkolationswahrscheinlichkeit $\Pi$ als Funktion der Besetzungswahrscheinlichkeit $p$ berechnet. Dabei wird der Fall $p=0$ ausgelassen, da hier offensichtlich $\Pi=0$ ist.

In [ ]:
def percolation(n_edge, n_p, n_ensemble, n_total,
                neighbours_fct):
    dp = 1 / n_p
    p_values = np.linspace(dp, 1, n_p)
    probabilities = np.zeros(n_p)
    for m, p in enumerate(p_values):
        probabilities[m] = probability(
            p, n_edge, n_ensemble, n_total, neighbours_fct)
    return p_values, probabilities

### Implementierung der Bedienelemente und graphische Darstellung der Perkolationswahrscheinlichkeit

Mit Hilfe der Bedienelemente können die folgenden Parameter eingestellt werden:
- `n_edge`: Seitenlänge des Gitters
- `n_p`: Anzahl der äquidistanten Besetzungswahrscheinlichkeiten
- `n_ensemble`: Größe des Ensembles zur Berechnung der Perkolationswahrscheinlichkeit
- `percolation_type`: Auswahl zwischen Knoten- und Kantenperkolation auf dem Quadratgitter

Der Verlauf der Perkolationswahrscheinlichkeit als Funktion der Besetzungwahrscheinlichkeit wird graphisch für die gewählte Ensemblegröße dargestellt. Außerdem wird die entsprechende Kurve für ein System mit einem Fünftel der gewählten Kantenlänge gezeigt, um den Einfluss der Systemgröße zu verdeutlichen. Da die Rechnung etwas länger dauern kann, wird am Ende die benötigte Rechenzeit angegeben. Durch Vergleich der Angabe für verschiedene Systemgrößen lässt sich der Rechenzeitbedarf für größere System abschätzen.

In [ ]:
widget_dict = {"n_edge":
               widgets.IntSlider(
                   value=100, min=20, max=100, step=1,
                   description=r"$n_\text{edge}$"),
               "n_p":
               widgets.IntSlider(
                   value=50, min=10, max=200, step=5,
                   description="$n_p$"),
               "n_ensemble":
               widgets.IntSlider(
                   value=200, min=10, max=200, step=5,
                   description=r"$n_\text{Ensemble}$"),
               "percolation_type":
               widgets.RadioButtons(
                   value="Knotenperkolation",
                   options=["Knotenperkolation",
                            "Kantenperkolation"],
                   description="Perkolationstyp")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_transition(n_edge, n_p, n_ensemble,
                    percolation_type):
    n_edge_reduced = n_edge // 5
    if percolation_type == "Knotenperkolation":
        n_total = n_edge**2
        n_total_reduced = n_edge_reduced**2
        neighbours_fct = neighbours_sites
    else:
        n_total = 2*n_edge**2
        n_total_reduced = 2*n_edge_reduced**2
        neighbours_fct = neighbours_bonds

    start_time = time.time()
    fig, ax = plt.subplots()
    for n_edge_val, n_total_val in (
            (n_edge, n_total),
            (n_edge_reduced, n_total_reduced)):
        result = percolation(n_edge_val, n_p, n_ensemble,
                             n_total_val, neighbours_fct)
        ax.plot(*result)
    ax.set_xlabel("$p$")
    ax.set_ylabel(r"$\Pi$")
    end_time = time.time()
    print(f"benötigte Zeit: {end_time-start_time:6.1f}s")